In [1]:
from pixiedust.display.app import *
import requests
import re

CLOUDANT_CONN_TYPE = "cloudant"
BASELINE_PREFIX = "baseline"

@PixieApp
class SelectBaseline():    
    def getExistingBaselines(self):
        conn = json.loads( getConnection(CLOUDANT_CONN_TYPE, self.connection)["PAYLOAD"])["credentials"]
        url = conn['url'] + "/_all_dbs"
        listDbs = requests.get(url).json()
        return [db for db in listDbs if re.match("{}-([\\w\\$])+$".format(BASELINE_PREFIX),db)]
                                                 
    @route(connection="*", existing_baselines="true")
    def selectBaseline(self):
        return """
<div>
  <div class="form-horizontal">
    <div class="form-group">
      <label class="control-label col-sm-2">Select a baseline to work with:</label>
      <div class="col-sm-5">
        <select class="form-control" id="baseline{{prefix}}">
          {%for baseline in this.getExistingBaselines() %}
              <option value="{{baseline|escape}}">{{baseline|escape}}</option>
          {%endfor%}
        </select>
      </div>
      <div class="col-sm-1">
        <button type="submit" class="btn btn-primary" pd_script="self.workingBaseline = $val(baseline{{prefix}})">Go</button>
      </div>
    </div>
  </div>
</div>
"""

Pixiedust database opened successfully


In [3]:
from pixiedust.display.app import *
from pixiedust.utils import Logger
from pixiedust.utils.shellAccess import ShellAccess
from requests.auth import HTTPBasicAuth
import base64
import json
import time
import pandas as pd
from pandas.io.json import json_normalize
from pixiedust.services.serviceManager import *
headers={
    "X-Watson-Origin" : "anything"
}
@PixieApp
@Logger()
class ConversationLogger(SelectBaseline):
    def fetchLogs(self):
        ws_id = "7ff0c3c2-a08a-4d3d-b13b-b6a9b62a4296"
        response = requests.get("https://gateway.watsonplatform.net/conversation/api/v1/workspaces/"+ ws_id + "/logs?version=2017-04-21", 
                                headers = headers,
                                auth=HTTPBasicAuth('50c95e08-34ef-453f-99f7-66d93df2641c','TlZczNCmdtlG')
                                ).json()['logs']
        
        self.debug("json log: {}".format(json.dumps(response[0], indent=2)))
        selectedCols = {'request.input.text':'input', 'response.intents':'intents', 'request_timestamp':'inputDate'}
        pdf = json_normalize(response)[selectedCols.keys()]
        pdf.columns= selectedCols.values()
        pdf=pdf[ (pd.notnull(pdf['input'])) & (pdf['input']!="")]
        pdf.drop_duplicates(subset='input',inplace=True)
        pdf['intent'] = pdf.apply (lambda row: row['intents'][0]['intent'], axis=1)
        pdf['confidence'] = pdf.apply (lambda row: row['intents'][0]['confidence'],axis=1)
        del pdf['intents']
        self.log = ShellAccess.sqlContext.createDataFrame(pdf)
        
    def getConnections(self):
        return getConnections(CLOUDANT_CONN_TYPE)
    
    def getNewBaselineName(self):
        return "{}-{}".format(BASELINE_PREFIX, time.strftime('%Y%m%d$%H%M%S'))

    @route(save='true')
    def saveToCloudant(self):
        self.debug("Saving to Cloudant")
        print("Saving")
        
    @route()
    def selectConnection(self):
        return """
<div>
  <div class="form-horizontal">
    <div class="form-group">
      <label for="connection{{prefix}}" class="control-label col-sm-2">Select a cloudant connection:</label>
      <div class="col-sm-5">
        <select class="form-control" id="connection{{prefix}}">
          {%for conn in this.getConnections() %}
              <option value="{{conn.name|escape}}">{{conn.name|escape}}</option>
          {%endfor%}
        </select>
      </div>
      <div class="col-sm-1">
        <button type="submit" class="btn btn-primary" pd_options="existing_baselines=true" pd_script="self.connection = $val(connection{{prefix}})">Work with Existing Baselines</button>
        <button type="submit" class="btn btn-primary" pd_options="new_baseline=true">Create a new Baseline
            <pd_script>
self.connection = $val(connection{{prefix}})
self.newBaselineDbName = self.getNewBaselineName()
            </pd_script>
        </button>
      </div>
    </div>
  </div>
</div>        
        """
    
    @route(connection="*", new_baseline="true")
    def newBaseline(self):
        return """
<div>
    <button pd_entity="log" pd_options="handlerId=dataframe;table_noschema=true" pd_target="json{{prefix}}" type="submit" class="btn btn-primary">Fetch Logs for last 90 days
        <pd_script>
            self.fetchLogs()
        </pd_script>
    </button>
    <button pd_target="json{{prefix}}" pd_entity="log" pd_options="handlerId=stashCloudant;nostore_dbName={{this.newBaselineDbName}};nostore_connection={{this.connection}}" type="submit" class="btn btn-primary">Save to Cloudant
    </button>
</div>
<div id="json{{prefix}}">
</div>        
        """
    
a = ConversationLogger()
a.run()

In [10]:
display(a.log)